# **Illustration of the use of the stationary phase approximation.**

<p style="text-align: justify;font-size:15px;width: 90%">
In this notebook, we investigate the use of the stationary phase approximation applied to the evaluation of integrals of the form $\int \limits_{x_i}^{x_f} dx e^{i\alpha f(x)}$. To demonstrate its usefulness, we consider both the brute-force numerical evaluation of this integral and see that when $\alpha$ is increased, the number of discrete grid points required for accurate evaluation in this way becomes unfavourably large. On the other hand, we observe that with increasing $\alpha$ the stationary phase approximation becomes an increasingly better estimation of the integral. This, we see, is due to the more pronounced cancellations of the contributions to the integral from the highly oscillatory parts of the integrand lying outwith the immediate vicinity of the point where the integrand's phase is stationary.
    
 At the heart of the stationary phase approximation is the realization that only contributions to the integral coming from the region near that in which the phase of the integrand is stationary shall contribute non-negligibly. As illustrated by the plots within this notebook, the parts of the integral outwith this region undergo strong cancellations due to the highly oscillatory integrand. As a consequence, we can recover the leading asymptotic behaviour of the integral by expanding the argument of the exponential, $f(x)$ in our integrand to second order about its stationary point. Supposing that this stationary point is located at $x=c$ so that $f'(c)=0$ then we find
$$\int \limits_{x_i}^{x_f} dx e^{i \alpha f(x)}$$
$$ \simeq \int \limits_{x_i}^{x_f} dx e^{i \alpha\{f(c)+\frac{1}{2}f''(c)(x-c)^2 \}}$$.

A further consequence of the cancellations due to the large oscillations far from the stationary point is that we have a good justification for extending the limits of our integral to $\pm \infty$. There shall be negligible error introduced by doing this due to the aforementioned strong oscillations. Hence, we arrive at the following:
$$\int \limits_{-\infty}^{\infty} e^{i \alpha f(c)}  dx e^{\frac{\alpha}{2}f''(c)(x-c)^2 }$$. 

Now, this integral looks very much like one which we know how to solve - namely, a Gaussian integral. In fact, if we analytically continue this integral to the complex plane and deform our contour so that we pass through the stationary point at a polar angle corresponding to the path of steepest descent, we obtain the generalized Gaussian integral result:
$$  e^{i \alpha f(c)}\sqrt{\frac{\pi}{2i\alpha|f''(c)|}}$$. 
</p>

# ** Difficulty of converging a highly oscillatory integral with naive numerical integration.**
<p style="text-align: justify;font-size:15px;width: 90%">

Let us first look at how easy (or rather how difficult) it is to evaluate, using standard brute force numerical integraion, integrals with such highly oscillatory integrands. Below are plots of the integrand $F(x)=e^{i\alpha(x^2-a)}$ along with plots of the numerical estimate of $\int  \limits_{-2}^{2}dx e^{i\alpha(x^2-a)}$ using a very basic numerical quadrature approach wherein the integral is approximated by evaluating the sum $\sum_n F(n dx) \cdot dx$ where $dx=\frac{x_f-x_i}{N_{\text{grid}}}$ with $x_f=2$,$x_i=-2$ and $N_{\text{grid}}$ is the number of gridpoints used in the discretization of the integration range. 
    </p>

In [3]:
%matplotlib widget

from ipywidgets import interactive,Dropdown,Layout,VBox,HBox,FloatSlider
import matplotlib.pyplot as plt
import numpy as np

x=np.linspace(-2,2,2000)

def f1_x(x,alpha,a):
    return np.exp(1.j*alpha*(x**2-a))

def int_f1(x,alpha,a,Ngrid):

    dx=(x[-1]-x[0])/Ngrid
    I=0.+0.j
    for n in range(Ngrid):
        I += f1_x(x[0]+n*dx,alpha,a)*dx

    return I
    
def f1_int_x_interactive(integral_lims=2.0,alpha=10.0,a=0.0, Ngrid=1000,Real_or_Imaginary_part='real'):
    
    axs1.clear()
    x = np.linspace(-2, 2, num=2000)

    f = f1_x(x,alpha,a)
    if(Real_or_Imaginary_part=='real'):
        re_f=np.real(f)
        axs1.plot(x, re_f,label="Re(f(x))" )
        axs1.fill_between(x, re_f, 0,where=re_f>0.,facecolor='yellow', alpha=0.5)
        axs1.fill_between(x, re_f, 0,where=re_f<0.,facecolor='green', alpha=0.5)
    else:
        im_f = np.imag(f)
        axs1.plot(x, np.imag(f),label="Im(f(x))",color='red')
        axs1.fill_between(x, im_f, 0,where=im_f>0.,facecolor='yellow', alpha=0.5)
        axs1.fill_between(x, im_f, 0,where=im_f<0.,facecolor='green', alpha=0.5)

    axs2.clear()
    axs2.set_xlim(0,Ngrid)

    If_lst1=[]
    grid_list=np.arange(1,Ngrid)
    for n_points in grid_list:
        If_lst1.append(int_f1(x, alpha,a, n_points))  
    If_lst1=np.array(If_lst1)

    if(Real_or_Imaginary_part=='real'):
        axs2.plot(grid_list, np.real(If_lst1),label=r"Re($\int _{L}^{L} f(x)$)" )

    else:

        axs2.plot(grid_list, np.imag(If_lst1),label=r"Im($\int_{L}^{L} f(x)$)",color='red' )

    axs1.set_xlabel('x')
    axs2.set_xlabel('Ngrid')

    axs1.set_title(r"Plot of integrand $\exp(i \alpha (x^2-a))$.")        
    axs2.set_title("Plot of numerical integration vs number of grid points.")
    
    plt.legend()
    plt.show()
        
    
fig1=plt.figure()    
plt.subplots_adjust(hspace = .5)   
axs1=fig1.add_subplot(211)
line1, = axs1.plot([],[])
axs2=fig1.add_subplot(212)

line2, = axs2.plot([],[])

interactive_plot1 = interactive(f1_int_x_interactive,\
                                integral_lims=FloatSlider(value=2.0,min=1.0,max=10.0,step=0.1,continuous_update=False),\
                                Ngrid=1000, \
                                alpha=FloatSlider(value=10.0, min=1.0,max=1000.0,step=10.,continuous_update=False),\
                                a=FloatSlider(value=0.0, min=-1.0,max=1.0,step=.1,continuous_update=False),\
                                Real_or_Imaginary_part=['real', 'imaginary'])

# interactive_plot
box1 = VBox( [interactive_plot1], layout=Layout(width='400px'))
box1

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# ** Convergence of the stationary phase approximation to the exact result.**
<p style="text-align: justify;font-size:15px;width: 90%">
We now look at how good a job the stationary phase approximation does of estimating the true value of the integral depending on how large $\alpha$ (the strength of the oscillations) is. Furthermore, we see how much error is introduced by extending the integration limits to $\pm \infty$ from some finite interval $(x_i,x_f)$. For this example we consider the integrand $F(x)=e^{i\alpha( \cdot e ^{x^2}-1)}$ (the reason for this seemingly convoluted choice of integrand is simply to avoid some unnecessary complications in the visualization).
    </p>

In [9]:
"""
Trying out a function f(x) which should have 1 SP and have a 2nd order Taylor expansion which is different from 
f(x) itself: exp^(i*alpha*exp^(-x^2))
"""

from ipywidgets import interactive,Dropdown,Layout,VBox,HBox
import matplotlib.pyplot as plt
import numpy as np

def f_x(x,alpha):
    return np.exp(1.j*alpha*(np.exp(1.*x**2)-1))
    
def int_f(x,alpha,Ngrid):
    
    If_lst=[]
    dx=(x[-1]-x[0])/Ngrid
    I=0.+0.j
    for n in range(int(Ngrid/2)):
        I += 2.*f_x(n*dx,alpha)*dx
        If_lst.append(I)
        
    return If_lst
    
def f_int_x_interactive(integral_lims=2.0,alpha=10.0, Ngrid=1000,Real_or_Imaginary_part='real'):
    
    ax1.clear()
    x = np.linspace(-2, 2, num=2000)

    f = np.exp(1.j*alpha*np.exp(1.*x**2))
    f_sp = np.exp(1.j*alpha*(x**2))
    if(Real_or_Imaginary_part=='real'):

        ax1.plot(x, np.real(f),label="Re" )
        ax1.plot(x, np.real(f_sp),label="Re(sp approx)" )

    else:

        ax1.plot(x, np.imag(f),label="Im", color="red")
        ax1.plot(x, np.imag(f_sp),label="Im(sp approx)", color="green")

    ax2.clear()
    ax2.set_xlim(0,integral_lims)
    x = np.linspace(0, integral_lims, num=int(Ngrid/2))    

    If_lst=np.array(int_f(x, alpha, Ngrid)) 

    If_sp = np.sqrt(np.pi/alpha)*np.exp(1.j*np.pi/4.)*np.ones(len(x))
    if(Real_or_Imaginary_part=='real'):
        ax2.plot(x, np.real(If_lst),label=r"Re($\int _{L}^{L} f(x)$)" )
        ax2.plot(x, np.real(If_sp),label="Re(SP approx)" )

    else:

        ax2.plot(x, np.imag(If_lst),label=r"Im($\int_{L}^{L} f(x)$)", color="red" )
        ax2.plot(x, np.imag(If_sp),label="Im(SP approx)", color="green")

    ax1.set_xlabel('x')
    ax2.set_xlabel('L')

    ax1.set_title(r"Plot of integrand $\exp(i \alpha (e^{-x^2}-1))$.")        
    ax2.set_title("Plot of numerical integration and stationary phase approx")
    
    plt.legend()
    plt.show()
          
fig=plt.figure()    
plt.subplots_adjust(hspace = .5)   
ax1=fig.add_subplot(211)
line1, = ax1.plot([],[])
ax2=fig.add_subplot(212)
line2, = ax2.plot([],[])
interactive_plot = interactive(f_int_x_interactive,integral_lims=(1.0,10.0),Ngrid=1000, alpha=(0.0,100,1.), Real_or_Imaginary_part=['real', 'imaginary'])

# interactive_plot
box1 = VBox( [interactive_plot], layout=Layout(width='400px'))
box1

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<!-- # ** Convergence of the stationary phase approximation to the exact, numerically integrated result with increasing $\alpha$.  **

<p style="text-align: justify;font-size:15px;width: 90%">
    We now look at how the error introduced by the stationary phase approximation becomes less and less significant as we increase the value $\alpha$ which controls the strength of the oscillations in the integrand.
</p> -->